In [ ]:
import os
import sys
import pandas as pd
sys.path.append('/home/mark/Documents/code/mdrp')


import numpy as np
from utils.nlp_mdrp import mdrp_prob
import cyipopt

In [ ]:
num_i = 5
num_j = 3

prob_setup = {}

prob_setup['s_time']   = np.array([2,4,3])/60
prob_setup['t_time']   = np.array([[45,30,10,10,10],
                                   [15,10,10,8,5],
                                   [20,25,30,20,20]]).T/60
prob_setup['cost']     = np.array([[12,12,12,12,12],
                                   [5,5,5,5,5],
                                   [5,5,5,5,5]]).T
prob_setup['beta']     = np.array([[0.5,0.5,0.5,0.5,0.5],
                                   [0.5,0.5,0.5,0.5,0.5],
                                   [0.5,0.5,0.5,0.5,0.5]]).T
prob_setup['N']        = np.ones(num_j)*10
prob_setup['k']        = np.ones(num_j)*10/60
prob_setup['mu']       = np.array([2,4,3])
prob_setup['max_rho']  = 0.9
prob_setup['max_cost'] = 10000
prob_setup['demand'] = 0.99

# for param in prob_setup.keys():
#     print('Parameter: %s \nValue:\n'%param,prob_setup[param])
#     print('-----------------------------')

# mdrp = mdrp_prob(prob_setup)

# max_demand = mdrp._get_max_demand()
# print('Max Demand: ',max_demand)


In [ ]:
from utils.convert import convert_instance

instance_dir = '/home/mark/Documents/code/mdrp/instances/0o50t75s1p100'
parameters = {}
parameters['speeds'] = [1,2,0.2]
parameters['N_ratios'] = [1,1,1]
parameters['u_ratios'] = [1,0.5,0.5]

prob_setup = convert_instance(instance_dir, parameters)
for param in ['N','k','mu']:
    print('Parameter: %s \nValue:\n'%param,prob_setup[param])
    print('-----------------------------')

In [ ]:
def traveltime(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    locations.set_index('id',inplace=True)

    orders.set_index('order',inplace=True)
    couriers.set_index('courier',inplace=True)
    restaurants.set_index('restaurant',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']
    return orders,restaurants,couriers,instanceparams,locations,meters_per_minute,\
           pickup_service_minutes,dropoff_service_minutes,target_click_to_door,\
           pay_per_order,guaranteed_pay_per_hour


Here are the paramters required and how we plan to load them using the instances

- s_time: (service time) Listing 4 pickup+dropoff time,
- t_time: (travel time) Listing 4 Travel time multiplier * Listing1&2 distance b/w resteraunt + order
- cost: (cost) Listing 4 per-order pay (constant for all orders), can be set seperately for different modes
- beta: (fraction of available couriers): compute using function based on Listing 2 Courier locations and Listing 4 Resteraunt locations
- N: (number of couriers per type) Listing 3 courier set, divided by the courier types
- k: (minutes around pickup location) This can be done together with computing N, beta, & mu.
- mu: (rate of order completioono) pre compute as ivnerse of average t_time + s_time

List any assumptions we are going to make:
- service time is constant
- travel time relies on L1 distance for cars+couriers, and l1 distance for drones
- drones are distributed uniformly around the grid, droids are distributed closer to the center. 

In [ ]:
instance_dir = '/home/mark/Documents/code/mdrp/instances/0o50t75s1p100'
print('reading instance information')   
orders,restaurants,couriers,instanceparams,locations,meters_per_minute,\
pickup_service_minutes,dropoff_service_minutes,target_click_to_door,\
pay_per_order,guaranteed_pay_per_hour = read_instance_information(instance_dir)

parameters = {}
# 800 meters per minute is roughly 30mph
parameters['speeds'] = [1,2,0.2]
parameters['N_ratios'] = [1,1,1]
parameters['u_ratios'] = [1,0.5,0.5]
n_modes = len(parameters['speeds'])


In [ ]:
# service time
s_t = (pickup_service_minutes+dropoff_service_minutes)/60
s_time = np.array([s_t, s_t/2, s_t/2])
t_time = np.array

# travel time
n_orders = len(orders)
t_time = np.zeros((n_orders,n_modes))
temp = []
for order in orders.index:
    temp.append(traveltime(order,orders.at[order,'restaurant'],meters_per_minute,locations)/60)
for j in range(n_modes):
    t_time[:,j] = np.asarray(temp)/parameters['speeds'][j]

# costs just simple constant model
cost = np.zeros_like(t_time)
mode_costs = [10,5,5]
for j in range(n_modes):
    cost[:,j] = mode_costs[j]*np.ones(n_orders)



In [ ]:
# beta, N, k, mu All together.
car_locs = np.array([couriers.x,couriers.y]).T
N = np.array(parameters['N_ratios'])*len(car_locs)
# print(N)
k = np.ones(num_j)*10/60 # 5 minutes around center

# lets make drone couriers (unifromly around entire grid)
n_drones = len(car_locs)*parameters['N_ratios'][1]
grid_x = [restaurants.x.min(),restaurants.x.max()]
grid_y = [restaurants.y.min(),restaurants.y.max()]
drone_locs = np.array([grid_x[0]+np.random.rand(n_drones)*(grid_x[1]-grid_x[0]),
                       grid_y[0]+np.random.rand(n_drones)*(grid_y[1]-grid_y[0])]).T

n_droids = len(car_locs)*parameters['N_ratios'][2]
grid_x = [restaurants.x.mean()-0.5*restaurants.x.std(),restaurants.x.mean()+0.5*restaurants.x.std()]
grid_y = [restaurants.y.mean()-0.5*restaurants.y.std(),restaurants.y.mean()+0.5*restaurants.y.std()]
droid_locs = np.array([grid_x[0]+np.random.rand(n_droids)*(grid_x[1]-grid_x[0]),
                       grid_y[0]+np.random.rand(n_droids)*(grid_y[1]-grid_y[0])]).T

all_locs = [car_locs, drone_locs, droid_locs]
# beta
beta = np.zeros_like(t_time)
for i in range(1,n_orders+1):
    rest = orders.at['o%d'%i,'restaurant']
    loc = [restaurants.at[rest,'x'],restaurants.at[rest,'y']]
    for j in range(n_modes):
        count = 0
        for cour_loc in all_locs[j]:
            dist = np.sqrt(((cour_loc-loc)**2).sum())
            # print((dist/(meters_per_minute*parameters['speeds'][j]))/60)
            if (dist/(meters_per_minute*parameters['speeds'][j]))/60 < k[j]:
                count+=1
        beta[i-1,j] = count/N[j]

mu = 1/(((k/(1+beta*N*(0.1))) + s_time + t_time).mean(axis=0))

mu

In [ ]:
5/60